In [17]:
import requests 
import json 
from bs4 import BeautifulSoup as Soup
import re 


In [18]:
def get_all_documents(year):
    links = []
    base_url = f"https://www.retsinformation.dk/api/documentsearch?o=80&ps=100&yl={year}&yh={year}"
    tmp_request = requests.get(base_url)
    tmp_json = json.loads(tmp_request.content)
    if int(tmp_json['pageCount']) != 0:
        for i in tmp_json['documents']:
            links.append(i['retsinfoLink'])
    if int(tmp_json['pageCount']) > 1: 
        for i in range(1,int(tmp_json['pageCount'])):
            print(f"{year} - {i}")
            tmp_request = requests.get(base_url+f"&page={i}")
            tmp_json = json.loads(tmp_request.content)
            for i in tmp_json['documents']:
                links.append(i['retsinfoLink'])
    return links 

In [19]:
def get_accn(links): 
    accn_codes = []
    base_url = "https://www.retsinformation.dk/api/document"
    for link in links: 
        tmp_url = base_url + link 
        tmp_request = requests.get(tmp_url)
        try: 
            tmp_json = json.loads(tmp_request.content) 
        except json.JSONDecodeError:
            with open("errors.txt","a") as outfile:
                outfile.write(link + "\n")
        for meta in tmp_json[0]['metadata']:
            if meta['displayName'] == 'Accession':
                accn_codes.append(meta['displayValue'])
                try: 
                    with open(f"./files/{meta['displayValue']}.json","w") as outfile:
                        json.dump(tmp_json[0],outfile,indent=4,ensure_ascii=False)
                    print(f"Finished {meta['displayValue']}")
                except UnicodeEncodeError as e: 
                    with open("errors.txt","a") as outfile:
                        outfile.write(meta['displayValue'] + "\n")
                    print(f"Error on {meta['displayValue']}")
    return accn_codes


In [20]:
for i in range(1982,2023):
        print(i)
        tmp_links = get_all_documents(i)
        if len(tmp_links) != 0:
            get_accn(tmp_links)

1982
1982 - 1
